In [ ]:
!pip install boto3 langchain langchain_community -Uq

In [ ]:
import json
import boto3
import random
import pandas as pd

In [ ]:
bedrock_runtime = boto3.client('bedrock-runtime')

In [ ]:
filename = '游戏聊天日志03.06~03.09.csv'
df = pd.read_csv(filename)
# with open(filename) as f:
#     lines = f.readlines()

In [ ]:
df.shape

In [ ]:
df.channelid.describe()

In [ ]:
df.groupby('channelid').count()

In [ ]:
df = df.sort_values(by=['channelid', '@timestamp'], ascending=[True, True])

In [ ]:
df

In [ ]:
c_id = 0

In [ ]:
%%time
df['chat'] = df.apply(lambda x: f"<channel>{'世界频道_'+str(x['channelid']) if x['channelid'] == 0 else '其他频道_'+str(x['channelid']) }<channel><username>{x['senderrolename']}</username><message>{x['_c3']}</message>", axis=1)

In [ ]:
df_new = df[df.channelid==c_id]
df_new

In [ ]:
extracted_data = list(df_new.chat.values)
len(extracted_data)

In [ ]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate

In [ ]:
llm_sonnet = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 4000,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

llm_haiku = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0",
                  model_kwargs={"temperature": 0,
                                "top_k":10,
                                "max_tokens": 4000,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=True,callbacks=[StreamingStdOutCallbackHandler()])

In [ ]:
template_extract = \
"""You are a great linguist classifer, and now you are asked to analyze the group chat history of users in a game, and identify the user if it is against the rules.
The rules is defined as classes [A,B,C,D]
Below is the definition of classes:
- A: "小号脚本广告",特征有：小号高频发言、文案内容重复、点对点私聊拉人；表现形式有：私聊询问要不要做拖、或者高福利玩XXX游戏；私聊询问要不要福利；私聊免费送福利； 私聊要不要一起玩新游；私聊给激活码；
- B: "诱导加微拉人",特征有：小号在世界频道或私聊以各种理由，诱导玩家加微信或者加QQ；表现形式有：小号在世界频道重复发言，说有福利码，要的私聊，私聊后要求加微信或者加QQ才给福利码；小号在世界频道收月卡号、收金币，私聊后，要求加微或者加QQ详聊；小号在世界频道说找队友一起玩游戏或者说不会玩找人教，私聊小号后，小号要求加微或者加QQ；小号在世界频道出号、免费送号、送金币，私聊后要求加微或QQ详聊；小号改名为【花的票帮找回】【氪多帮追回】等名字，在世界频道重复发言，称充值可退款，私聊要求加微信或者加QQ；
- C: "诱导关注公众号拉人", 特征有：小号在世界频道或私聊诱导玩家关注公众号；表现形式有：小号在世界频道刷屏，说免费送礼包金币兑换码，诱导关注公众号；小号把名字改为带公众号的ID，说白嫖金币兑换码，诱导关注公众号；
- D: "工作室家族建群拉人；", 特征有：族长/管理/小号在家族/帮会频道高频发言、文案内容重复、私聊拉人. 表现形式:族长/管理在家族/帮会/私聊频道刷屏，说马上清理机器人/一起下副本/组固定队，要求玩家进群；族长/管理在家族/帮会频道刷屏，说让玩家注意查看邮件进群；族长不发言，家族内未充值小号在家族/帮会/私聊频道以族长名义拉人进群；

Here is the tasks:
Only find the quotes from the message that are against the rules and then print them in xml tag <response>. 
Quotes should be exactly the same as original text.
If there are no relevant quotes at all, write "No relevant quotes" instead.

Here is a chat history you will analyze
<doc>
{context}
</doc>


please enclose your analysis results in xml tag <response>.

the output format is json, for example:
<response>
[
{{"channel":xxx,"username":xxx,"message":xxx}}
]
</response>


Skip the preamble, go straight into the answer.
"""

In [ ]:
template_sentiment = \
"""You are a INFORMATION classifier
Now you are asked to analyze the group chat history of users in a game, and identify the user if it is against the rules.
Give you a list of sentences you will classify them to the predefined classes [A,B,C,D,N], output the class in <label_class>. and output a confidence score between 0 to 10, that represents how relevant it is to the class label, 10 means most relevant, and 0 is not relevant
Below is the definition of classes:
- A: "小号脚本广告",特征有：小号高频发言、文案内容重复、点对点私聊拉人；表现形式有：私聊询问要不要做拖、或者高福利玩XXX游戏；私聊询问要不要福利；私聊免费送福利； 私聊要不要一起玩新游；私聊给激活码；愿不愿意做内部玩家，福利玩家。
- B: "诱导加微拉人",特征有：小号在世界频道或私聊以各种理由，诱导玩家加微信或者加QQ；表现形式有：小号在世界频道重复发言，说有福利码，要的私聊，私聊后要求加微信或者加QQ才给福利码；小号在世界频道收月卡号、收金币，私聊后，要求加微或者加QQ详聊；小号在世界频道说找队友一起玩游戏或者说不会玩找人教，私聊小号后，小号要求加微或者加QQ；小号在世界频道出号、免费送号、送金币，私聊后要求加微或QQ详聊；小号改名为【花的票帮找回】【氪多帮追回】等名字，在世界频道重复发言，称充值可退款，私聊要求加微信或者加QQ；
- C: "诱导关注公众号拉人", 特征有：小号在世界频道或私聊诱导玩家关注公众号；表现形式有：小号在世界频道刷屏，说免费送礼包金币兑换码，诱导关注公众号；小号把名字改为带公众号的ID，说白嫖金币兑换码，诱导关注公众号；
- D: "工作室家族建群拉人；", 特征有：族长/管理/小号在家族/帮会频道高频发言、文案内容重复、私聊拉人. 表现形式:族长/管理在家族/帮会/私聊频道刷屏，说马上清理机器人/一起下副本/组固定队，要求玩家进群；族长/管理在家族/帮会频道刷屏，说让玩家注意查看邮件进群；族长不发言，家族内未充值小号在家族/帮会/私聊频道以族长名义拉人进群；
- E: "其他"， 不符合A,B,C,D分类特征。

Here are the sentences:
<sentences>
{relevant_info}
</sentences>

Please follow below requirements:
1. You will strictly be based on the document in <doc>.
2. please enclose your analysis results in xml tag <output>.
3. output a score between 0 -10,

the output format for example:
<output>
<channel>xxx</channel><username>xxx</username><message>xxx</message><label_class>A</label_class><score>10<score>
</output>

Skip the preamble, go straight into the answer. 

"""

In [ ]:
from langchain_core.output_parsers.base import BaseOutputParser
import re
class CustOuputParser(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<response>(.*?)</response>"
        match = re.search(pattern, content, re.DOTALL)
        if match:
            text = match.group(1)
            text = text.replace('[','(').replace(']',')') ##避免跟sentiment的格式冲突
            return text
        else:
            return 'No relevant quotes'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"


class CustOuputParser2(BaseOutputParser[str]):

    def extract(self,content:str) -> str:
        pattern = r"<output>(.*?)</output>"
        match = re.search(pattern, content, re.DOTALL)
        return match.group(1) if match else 'No sentiment'    
    
    def parse(self, text: str) -> str:
        cleaned_text = self.extract(text)
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"
    

In [ ]:
def split_into_chunks(arr, n):
    chunks = []
    for i in range(0, len(arr), n):
        chunks.append(arr[i:i+n])
    return chunks

In [ ]:
chunks = split_into_chunks(extracted_data,1000)

In [ ]:
print(f"total chunks {len(chunks)}")

In [ ]:
text =  "\n".join(chunks[0])
len(text)

In [ ]:
# from langchain_core.output_parsers import StrOutputParser
# prompt = ChatPromptTemplate.from_template(template)
prompt_extract = ChatPromptTemplate.from_template(template_extract)
prompt_sentiment = ChatPromptTemplate.from_template(template_sentiment)

In [ ]:
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
## 如果没有相关内容则无须invoke chain2
def route(info):
    if not 'no relevant quotes' in info['relevant_info'].lower():
        return chain_2

In [ ]:
output_parser = CustOuputParser()
output2_parser  = CustOuputParser2()

chain_1 = prompt_extract | llm_sonnet | output_parser
chain_2 = prompt_sentiment | llm_sonnet| output2_parser
# chain_3 = ({'relevant_info':chain_1,'topic':itemgetter('topic')})|prompt_sentiment | llm_haiku|output2_parser


In [ ]:
##测试chain3
# chain_3.invoke({'topic':"auction house",
#                        'context':text})

In [ ]:
%%time
##测试加入了路由选择的chainfull
full_chain = ({'relevant_info':chain_1})| RunnableLambda(route)
answer = full_chain.invoke({'context':text})

In [ ]:
print(answer)

In [ ]:
# question = "is there any content relevant to auction house?"

#'please list all the content if it is relevant and classify the sentiment of each content into [positive,neutral,negative]'

# answer = chain_2.invoke({'topic':"auction house",
#                        'context':text})

In [ ]:
def extract_info(text):
    pattern = r'<channel>(.*?)</channel><username>(.*?)</username><message>(.*?)</message><label_class>(.*?)</label_class><score>(.*?)</score>'
    matches = re.findall(pattern, text)
    print(matches)
    result = []
    for match in matches:
        # info = {
        #     'channel': match[0],
        #     'username': match[1],
        #     'message': match[2],
        #     'label_class': match[3],
        #     'score': int(match[4])
        # }
        info = [match[0], match[1], match[2],  match[3], int(match[4])]
        result.append(info)
    return result

In [ ]:
ret= extract_info(answer)
ret

In [ ]:
#save a list to a local file, if the file is exist then append the list to the file
def save_list(list,file_name):
    with open(file_name,'a') as f:
        for item in list:
            f.write("%s\n" % item)
    f.close()
    return True

In [ ]:
#save a list to a csv file using pandas, if the file is exist then append the list to the file
import pandas as pd
import os
def save_to_csv(data, filename):
    new_data = pd.DataFrame(data, columns=['channel', 'username', 'message', 'label_class', 'score'])
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        #append data to df
        df = pd.concat([df, new_data], ignore_index=True)
    else:
        df = new_data
    df.to_csv(filename, index=False)
    return df

循环每个chunk进行输出,每次追加到csv文件保存，

In [ ]:
import time
all_result = []
filename = f'chlid_{c_id}_parsed_result'

##断点跳过
skip_num =9
for i,chunk in enumerate(chunks):
    if i < skip_num:
        continue
    t1 = time.time()
    print(f'--------chunk idx:{i}-------')
    text =  "\n".join(chunk)
    answer = full_chain.invoke({
                       'context': text})
    extract_ret = extract_info(answer)
    print(f'\nextract_ret:\n{extract_ret}')
    time.sleep(40)
    print(f'---time cost {time.time()-t1} s -----\nsleep 10s for token tpm\n\n')
    if extract_ret:
        all_result+=extract_ret
        save_to_csv(extract_ret,filename+f"chunk_{i}.csv")
        save_to_csv(extract_ret,filename+f"_all.csv")

In [ ]:
len(all_result)